In [1]:
import sqlite3
from PIL import Image
con = sqlite3.connect('food_image.db')
cur = con.cursor()


In [2]:
cur.executescript('''
    DROP TABLE IF EXISTS food_image;
    CREATE TABLE food_image(
        FOOD_NAME    TEXT,
        FOOD_IMAGE    TEXT
    );
''')


In [3]:
# cur.executescript('''
#     DROP TABLE IF EXISTS IMAGE;
#     CREATE TABLE IMAGE(
#         FOOD_NAME    TEXT,
#         IMAGE    BLOB
#     );
# ''')

In [4]:
import requests
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
    else: #bot 접근 못하게 막은데있어어 else뺌
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500:
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
        return resp

In [16]:
resp = download('https://www.gettyimagesbank.com/s/?lv=&st=union&mi=2&q=%25EA%25B9%2580%25EC%25B9%2598%25EC%25B0%258C%25EA%25B0%259C&ssi=go',headers = headers, method='GET')
dom = BeautifulSoup(resp.text,'lxml')

<Response [200]>

In [65]:
flist = ['멍게비빔밥', '새우초밥', '연어초밥', '약초비빔밥', '해물덮밥', '월남쌈', '김치찌개', '꽁치김치찌개']

In [66]:
from selenium import webdriver
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')

for search_list in flist:
    driver.get('https://www.gettyimagesbank.com/s/?lv=&st=union&mi=2&q='+search_list+'&ssi=go')
    time.sleep(3)
    if not driver.find_elements_by_css_selector('img[class=imgThumb]'):
        continue
    for _ in driver.find_elements_by_css_selector('img[class=imgThumb]'):
                cur.execute('''
            INSERT INTO food_image(FOOD_NAME, FOOD_IMAGE)
            VALUES(?,?)''' ,[search_list,_.get_attribute('src')]
        )
con.commit()
driver.close()

In [ ]:
# from selenium import webdriver
# from urllib.request import urlopen
# driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')

# for search_list in flist:
#     driver.get('https://www.gettyimagesbank.com/s/?lv=&st=union&mi=2&q='+search_list+'&ssi=go')
#     time.sleep(3)
#     if not driver.find_elements_by_css_selector('img[class=imgThumb]'):
#         continue
#     for _ in driver.find_elements_by_css_selector('img[class=imgThumb]'):
#         img = urlopen(_.get_attribute('src'))
#         break
#                 cur.execute('''
#             INSERT INTO IMAGE(FOOD_NAME, IMAGE)
#             VALUES(?,?)''' ,[search_list,urlopen(_.get_attribute('src'))]
#         )
# con.commit()
# driver.close()

In [63]:
driver.close()

In [4]:
cur.close()
con.close()

In [50]:
from selenium import webdriver
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
driver.get('https://pixabay.com/ko/photos/search/')

In [52]:
driver.close()

# 음식 사진 crawling

In [51]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time
flist = ['토마토','고추', '당근', '마늘', '무', '호박']
url = 'https://pixabay.com/ko/photos/search/'


for i in flist:
    driver.get(url + i + '/?cat=food')
    num_of_pagedowns = 4
    time.sleep(3)
    while num_of_pagedowns:
        
        try:
            body = driver.find_element_by_tag_name('body')
            body.send_keys(Keys.END)
            time.sleep(4)
            num_of_pagedowns -= 1
        
            for j,_ in enumerate(driver.find_elements_by_css_selector('img')):
                if _.get_attribute('srcset') !='':
                    cur.execute('''
                        INSERT INTO food_image(FOOD_NAME, FOOD_IMAGE)
                        VALUES(?,?)''' ,[i,_.get_attribute('srcset')]
                     )
            time.sleep(2)
            driver.find_element_by_xpath('//*[@id="content"]/div/a').click()
            time.sleep(3)
        except StaleElementReferenceException:
            pass
con.commit()


# 유기농 제품 crawling

In [20]:
from selenium import webdriver
driver = webdriver.Chrome('/Users/sohyun/Downloads/chromedriver')

In [22]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time
# url = 'http://lovinghut.co.kr/product/list.html?cate_no=' + category + '&page=' + page_num

for category in range(52,54):
    driver.get('http://lovinghut.co.kr/product/list.html?cate_no=' + str(category))
    num_of_pagedowns = 4
    time.sleep(3)
    while num_of_pagedowns:
        try:
            body = driver.find_element_by_tag_name('body')
            body.send_keys(Keys.END)
            time.sleep(4)
            num_of_pagedowns -= 1
        
            for j,_ in enumerate(driver.find_elements_by_css_selector('img')):
                if _.get_attribute('src') !='':
                    print(_.get_attribute('src'))
#                     cur.execute('''
#                         INSERT INTO food_image(FOOD_NAME, FOOD_IMAGE)
#                         VALUES(?,?)''' ,[category,_.get_attribute('src')]
#                      )
            time.sleep(2)
            driver.find_element_by_xpath('//*[@id="contents"]/div[3]/p[3]/a/img').click()
            time.sleep(3)
        except StaleElementReferenceException:
            pass
con.commit()


http://lovinghut.co.kr/upload/logo.gif
http://lovinghut.co.kr/upload/ban_TR.jpg
http://img.echosting.cafe24.com/skin/base_ko_KR/product/btn_compare.gif
http://img.echosting.cafe24.com/skin/base_ko_KR/common/btn_list_search.gif
http://lovinghut.co.kr/web/product/medium/20200626/9bbb3f86b46401682de697b24350e710.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/20200518/78bd732a45c1ce325331460e2a246be8.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/20200518/36014be5fe06ff6a188c0e7a71e8147c.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/20200131/ae11896be5f98d4fb0d90f7060efb26a.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_482.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/20200131/e2657902

http://lovinghut.co.kr/upload/logo.gif
http://lovinghut.co.kr/upload/ban_TR.jpg
http://img.echosting.cafe24.com/skin/base_ko_KR/product/btn_compare.gif
http://img.echosting.cafe24.com/skin/base_ko_KR/common/btn_list_search.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_457.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_72.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_58.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_339.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://img.echosting.cafe24.com/skin/base_ko_KR/common/btn_page_first.gif
http://img.echosting.cafe24.com/skin/base_ko_KR/common/btn_page_prev.gif
http://img.echosting.cafe24.com/skin/base_ko_KR/common/btn_page_next.gif
http://img.echosting.cafe24.com/skin/base_ko_KR/common/btn_page_las

http://lovinghut.co.kr/web/product/medium/lovinghutkr_101.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_6.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_202.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_13.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_210.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_12.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_211.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_305.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/p

http://lovinghut.co.kr/web/product/medium/lovinghutkr_197.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_3.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_101.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_6.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_202.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_13.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_210.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/product/medium/lovinghutkr_12.jpg
http://img.echosting.cafe24.com/design/common/icon_card.gif
http://lovinghut.co.kr/web/pro

In [7]:
for category in range(52,56):
    print(category)

52
53
54
55


In [19]:
driver.close()